In [56]:
from __future__ import print_function, division
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from glob import glob

class VisDroneDataset(Dataset):
    """ VisDrone dataset."""

    def __init__(self, csv_files, root_dir, transform=None):
        """
        Args:
            csv_files (list of strings): Path to the csv file with annotations.
            root_dir (string): Directory containing annotations and images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        header_names =  ["frame_index","target_id","bbox_left","bbox_top","bbox_width","bbox_height","score","object_category","truncation", "occlusion"]
        self.frames_objects = dict()
        self.seq_range=dict()
        curr = 0
        for csv_file in csv_files:
            seq_name = csv_file.split("/")[-1].split(".")[0]
            df = pd.read_csv(csv_file, header=None, names=header_names)
            seq_len = max(df['frame_index'])
            self.seq_range[seq_name] = range(curr, curr + seq_len)
            curr += seq_len
            self.frames_objects[seq_name] = df
        self.num_frames = curr
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return self.num_frames

    def index2file(self, index):
        seq = ""
        for s in self.seq_range.keys():
            if index in self.seq_range[s]:
                seq = s
                break
        frame = index - self.seq_range[s][0] + 1
        return seq, frame

    def __getitem__(self, idx):
        '''
        Args:
            idx (int):  unique id of the frame
        returns:
            sample (tuple):
                0: image (PIL Image): ndarray of frame image
                1: item (dict):
                    sequence (string): sequence name
                    frameID (int): frame number
                    gt (list): a list of objects(dict)
                        objects (dict):
                            classID (int):
                            bbox (list): ["bbox_left","bbox_top","bbox_width","bbox_height"]
                            filename (string): image name.
                    dim (tuple): (w, h)
        '''
        seq, frame = self.index2file(idx)
        frame_objects = self.frames_objects[seq]
        objs = frame_objects.loc[frame_objects['frame_index'] == frame]
        img_name = "{}.jpg".format(str(frame).zfill(7))
        item = dict()
        item['sequence'] = seq
        item['frameID'] = frame
        item['gt'] = list()
        img_path = os.path.join(self.root_dir+"/sequences/"+seq,
                                img_name)
        for index,row in objs.iterrows():
            obj = dict()
            obj['classID'] = row.iloc[7]
            obj["bbox"] = row.iloc[2:6].values.tolist()
            obj['filename'] = img_path
            item['gt'].append(obj)

        try:
            image = Image.open(img_path)
            # img_tensor = torch.from_numpy(image).float()
        except FileNotFoundError:
            print("Frame not found: seq {} frame {}, path: {}".format(seq, frame, img_path))
        item['dim'] = image.size

        sample = (image, item)

        if self.transform:
            sample = self.transform(sample)

        # return sample
        return sample


class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, item = sample[0], sample[1]

        w, h = image.size
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        # img = transform.resize(image, (new_h, new_w))
        image = image.resize((new_w, new_h))

        item['dim'] = image.size

        for obj in item['gt']:
            obj['bbox'] = [a*b for a,b in zip(obj['bbox'], [new_w / w, new_h / h, new_w / w, new_h / h])]

        return (image, item)
        # return {'image': img, 'item': item}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, item = sample[0], sample[1]

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        l = list()
        for i in range(len(item['gt'])):
            item['gt'][i]['bbox'] = torch.FloatTensor(item['gt'][i]['bbox'])
            
        norm = np.array(image)[:, :, ::-1].transpose(2, 0, 1).astype(np.float32) * (1.0 / 255.0)
        img_tensor = torch.FloatTensor(np.ascontiguousarray(norm))
        return (img_tensor, item)
        # return {'image': torch.from_numpy(image),
                # 'item': torch.from_numpy(np.array(l))}


In [57]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def show_bbox(image, items):
    """Show image with landmarks"""
    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Create a Rectangle patch
    for bbox in items['gt']:
        bbox = bbox['bbox']
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()



In [58]:
data_path = "/home/hu440/LP-MOT/LP-MOT/data/VisDrone"
visdrone_folders = [name for name in glob(os.path.join(data_path, '*'))]
train_csv = list()
val_csv = list()
test_csv = list()
for folder in visdrone_folders:
    for txt in glob(os.path.join(folder, 'annotations', '*')):
        if 'train' in folder:
            train_objs = open(txt).readlines()
            train_csv.append(txt)
        if 'val' in folder:
            val_objs = open(txt).readlines()
            val_csv.append(txt)
        if 'test' in folder:
            test_objs = open(txt).readlines()
            test_csv.append(txt)
# print(data_path)

# print(train_csv)

train_dataset = VisDroneDataset(
                        csv_files=train_csv,
                        root_dir=data_path+"/VisDrone2019-MOT-train",
                        transform=transforms.Compose([Rescale(400), ToTensor()]))
obj = train_dataset[0]


        

In [62]:
obj[0].size()

torch.Size([3, 400, 711])

In [54]:
train_dataset = VisDroneDataset(csv_files=train_csv, root_dir=data_path+"/VisDrone2019-MOT-train")

train_loader = DataLoader(
    train_dataset, 4, num_workers=2, pin_memory=True, drop_last=True, collate_fn=collate_fn, shuffle=False)
train_iter = iter(train_loader)
train_iter.next()

2
2


[[array([[[179, 182, 187],
          [182, 185, 190],
          [179, 182, 187],
          ...,
          [202, 200, 203],
          [188, 186, 189],
          [183, 181, 184]],
  
         [[181, 184, 189],
          [187, 190, 195],
          [188, 191, 196],
          ...,
          [188, 186, 189],
          [201, 199, 202],
          [210, 208, 211]],
  
         [[190, 193, 198],
          [193, 196, 201],
          [190, 193, 198],
          ...,
          [198, 196, 199],
          [171, 169, 172],
          [148, 146, 149]],
  
         ...,
  
         [[172, 169, 160],
          [164, 161, 154],
          [166, 166, 158],
          ...,
          [163, 159, 158],
          [161, 157, 156],
          [164, 160, 159]],
  
         [[175, 172, 163],
          [165, 162, 155],
          [167, 167, 159],
          ...,
          [171, 167, 166],
          [166, 162, 161],
          [161, 157, 156]],
  
         [[177, 174, 165],
          [169, 166, 159],
          [172, 172, 164

2
2
2


In [53]:
for i_batch, sample_batched in enumerate(train_loader):
    print(sample_batched[0], sample_batched[1])

    # observe 4th batch and stop.
    if i_batch == 3:
        break

2
2
2
2
2
2
[array([[[179, 182, 187],
        [182, 185, 190],
        [179, 182, 187],
        ...,
        [202, 200, 203],
        [188, 186, 189],
        [183, 181, 184]],

       [[181, 184, 189],
        [187, 190, 195],
        [188, 191, 196],
        ...,
        [188, 186, 189],
        [201, 199, 202],
        [210, 208, 211]],

       [[190, 193, 198],
        [193, 196, 201],
        [190, 193, 198],
        ...,
        [198, 196, 199],
        [171, 169, 172],
        [148, 146, 149]],

       ...,

       [[172, 169, 160],
        [164, 161, 154],
        [166, 166, 158],
        ...,
        [163, 159, 158],
        [161, 157, 156],
        [164, 160, 159]],

       [[175, 172, 163],
        [165, 162, 155],
        [167, 167, 159],
        ...,
        [171, 167, 166],
        [166, 162, 161],
        [161, 157, 156]],

       [[177, 174, 165],
        [169, 166, 159],
        [172, 172, 164],
        ...,
        [177, 173, 172],
        [168, 164, 163],
        [15

In [30]:
train_dataset.seq_range['uav0000248_00001_v']

range(0, 677)

In [33]:
train_dataset[677]['item']

{'sequence': 'uav0000295_02300_v',
 'frameID': 1,
 'gt': [{'classID': 4, 'bbox': [1141, 927, 75, 72], 'filename': '0000001.jpg'},
  {'classID': 9, 'bbox': [1040, 1013, 174, 335], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1120, 997, 86, 76], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1021, 1383, 132, 142], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1229, 1249, 112, 132], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1353, 941, 76, 74], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1365, 1117, 97, 95], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1381, 1221, 100, 119], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1384, 1345, 136, 180], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1481, 1052, 96, 97], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1536, 1313, 125, 148], 'filename': '0000001.jpg'},
  {'classID': 4, 'bbox': [1573, 985, 80, 76], 'filename': '0000001.jpg'},
  {'classID': 9, 'bbox': [1592, 1005